In [48]:
import openpyxl
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill

In [49]:
excel_file_path = "data/test.xlsx"

workbook = openpyxl.load_workbook(excel_file_path, data_only=True)

sheet = workbook['Лист1']

cell_info = {}

In [50]:
for row in sheet.iter_rows():
    for cell in row:
        cell_address = cell.coordinate

        cell_value = cell.value

        cell_data_type = type(cell_value).__name__

        borders = {
            "top": cell.border.top.style if cell.border.top.style else None,
            "right": cell.border.right.style if cell.border.right.style else None,
            "bottom": cell.border.bottom.style if cell.border.bottom.style else None,
            "left": cell.border.left.style if cell.border.left.style else None
        }

        # Определяем цвет ячейки
        # color_in_hex = cell.fill.start_color.index

        column_index = cell.column
        row_index = cell.row

        def get_value(sheet, column_index, row_index):
            if column_index < 1 or row_index < 1:
                return None
            return sheet[get_column_letter(column_index) + str(row_index)].value

        neighbor_cells = {
            "above": get_value(sheet, column_index, row_index - 1),
            "below": get_value(sheet, column_index, row_index + 1),
            "left": get_value(sheet, column_index - 1, row_index),
            "right": get_value(sheet, column_index + 1, row_index),
            "upper_left": get_value(sheet, column_index - 1, row_index - 1),
            "upper_right": get_value(sheet, column_index + 1, row_index - 1),
            "lower_left": get_value(sheet, column_index - 1, row_index + 1),
            "lower_right": get_value(sheet, column_index + 1, row_index + 1)
        }

        cell_info[cell_address] = {
            "value": cell_value,
            "data_type": cell_data_type,
            "borders": borders,
            # "color": color_in_hex,
            "neighbors": neighbor_cells
        }

for cell_address, info in cell_info.items():
    print(f"Ячейка {cell_address}:")
    print(f"Значение: {info['value']}")
    print(f"Тип данных: {info['data_type']}")
    print(f"Границы: {info['borders']}")
    # print("Цвет фона:", info["color"])
    print(f"Соседние ячейки: {info['neighbors']}")
    print()

Ячейка A1:
Значение: A
Тип данных: str
Границы: {'top': 'thin', 'right': 'thin', 'bottom': 'thin', 'left': 'thin'}
Соседние ячейки: {'above': None, 'below': 1, 'left': None, 'right': 'B', 'upper_left': None, 'upper_right': None, 'lower_left': None, 'lower_right': 1}

Ячейка B1:
Значение: B
Тип данных: str
Границы: {'top': 'thin', 'right': 'thin', 'bottom': 'thin', 'left': 'thin'}
Соседние ячейки: {'above': None, 'below': 1, 'left': 'A', 'right': 'C', 'upper_left': None, 'upper_right': None, 'lower_left': 1, 'lower_right': 1}

Ячейка C1:
Значение: C
Тип данных: str
Границы: {'top': 'thin', 'right': 'thin', 'bottom': 'thin', 'left': 'thin'}
Соседние ячейки: {'above': None, 'below': 1, 'left': 'B', 'right': 'D', 'upper_left': None, 'upper_right': None, 'lower_left': 1, 'lower_right': 1}

Ячейка D1:
Значение: D
Тип данных: str
Границы: {'top': 'thin', 'right': 'thin', 'bottom': 'thin', 'left': 'thin'}
Соседние ячейки: {'above': None, 'below': 1, 'left': 'C', 'right': None, 'upper_left': No